In [0]:
 !pip install kaggle

In [0]:
!pip install -U efficientnet

Requirement already up-to-date: efficientnet in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

train.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:

from zipfile import ZipFile
file_name = "train.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
from zipfile import ZipFile
file_name = "test.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:

import cv2
import numpy as np
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm

import tensorflow
import efficientnet.tfkeras as efn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,GlobalAveragePooling2D,Flatten,LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop


In [0]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'
IMG_SIZE = 100
LR = 1e-3


MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR,'2conv-basic')


In [0]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [0]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_COLOR),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [0]:
def create_test_data():
    testing_data = []
    # IMG = 1.jpg and TEST_DIR = /test
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_COLOR),(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    np.save('test_data.npy',testing_data)
    return testing_data

In [0]:
train_data = create_train_data() 

100%|██████████| 25000/25000 [00:57<00:00, 432.30it/s]


In [0]:
test_data = create_test_data()


100%|██████████| 12500/12500 [00:28<00:00, 435.33it/s]


In [0]:
train_data[0] 

[array([[[152, 131, 146],
         [161, 129, 154],
         [146,  97, 111],
         ...,
         [ 44,  38,  31],
         [ 46,  40,  33],
         [ 38,  32,  25]],
 
        [[160, 142, 154],
         [146, 109, 132],
         [174, 114, 127],
         ...,
         [ 44,  38,  31],
         [ 47,  41,  34],
         [ 41,  35,  28]],
 
        [[188, 167, 176],
         [153, 111, 129],
         [160,  89,  97],
         ...,
         [ 47,  41,  34],
         [ 43,  37,  30],
         [ 41,  35,  28]],
 
        ...,
 
        [[144, 154, 152],
         [146, 157, 155],
         [147, 158, 156],
         ...,
         [211, 194, 189],
         [204, 188, 182],
         [200, 184, 179]],
 
        [[146, 158, 158],
         [140, 153, 153],
         [142, 154, 154],
         ...,
         [206, 188, 187],
         [203, 185, 184],
         [203, 185, 184]],
 
        [[146, 158, 158],
         [149, 161, 161],
         [140, 153, 153],
         ...,
         [209, 191, 190],
  

In [0]:
train = train_data[:-500]
test = train_data[-500:]

In [0]:
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_train = [i[1] for i in train]

X_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_test = [i[1] for i in test]

X_train = X_train / 255
X_test = X_test / 255



In [0]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.1,1.5],

)
# keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False,
#      samplewise_center=False,
#       featurewise_std_normalization=False,
#        samplewise_std_normalization=False, 
#        zca_whitening=False,
#         zca_epsilon=1e-06, 
#         rotation_range=0, 
#         width_shift_range=0.0,
#          height_shift_range=0.0,
#           brightness_range=None,
#            shear_range=0.0,
#             zoom_range=0.0,
#              channel_shift_range=0.0,
#               fill_mode='nearest',
#                cval=0.0,
#                 horizontal_flip=False,
#                  vertical_flip=False,
#                   rescale=None,
#                    preprocessing_function=None,
#                     data_format='channels_last',
#                      validation_split=0.0,
#                       interpolation_order=1,
#                        dtype='float32')

In [0]:
type(X_train)


numpy.ndarray

In [0]:
train_generator = image_gen.flow(X_train,y_train,batch_size = 32)


val_generator = image_gen.flow(X_test,y_test,batch_size = 32)


In [0]:
model = Sequential()

model.add(LSTM(128, return_sequences = True, input_shape = (60, 5)))
model.add(Dropout(0.2))

model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(80, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(40))
model.add(Dropout(0.2))


model.add(Dense(units = 1))

In [0]:
model.compile(optimizer='adam', loss = 'mean_squared_error',metrics = ['accuracy'])

In [0]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
def model(input_shape, classes):
    pretrained_model = tensorflow.keras.applications.vgg16.VGG16(input_shape=input_shape, weights='imagenet', include_top=False)
    pretrained_model.trainable = False
    model = Sequential()
    for layer in pretrained_model.layers:
        model.add(layer)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation='softmax'))
    model.add(Dense(64, activation='softmax'))
    model.add(Dense(classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
    return model
# each pic has been resized to 100x100, and with 3 channels (RGB)
input_shape = (100,100,3)
classes = 2
model = model(input_shape, classes)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 25, 25, 256)      

In [0]:
model.fit_generator(train_generator,
                    epochs=20,
                    verbose=2,
                    validation_data = val_generator,
                    use_multiprocessing=True,
                    workers=2)


Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
766/766 - 1254s - loss: 0.5927 - acc: 0.8306 - val_loss: 0.4587 - val_acc: 0.8660
766/766 - 1254s - loss: 0.5927 - acc: 0.8306 - val_loss: 0.4587 - val_acc: 0.8660
Epoch 2/20
Epoch 2/20


In [0]:
with open('submission-file.csv','w') as f:
    f.write('id,label\n')

In [0]:
with open('submission-file.csv','w') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(-1,IMG_SIZE,IMG_SIZE,3)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))